In [1]:
!pip install --upgrade pip
!pip install google-api-python-client chromadb lark transformers sentencepiece llama-index sentence_transformers openai wikipedia langchain guidance japanize-matplotlib gradio
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 73.8 MB/s eta 0:0

In [15]:
from langchain.agents import load_tools, initialize_agent, Tool, AgentType
from langchain.utilities.google_search import GoogleSearchAPIWrapper
from langchain.chat_models import ChatOpenAI

import time
from tqdm import tqdm
import gradio as gr

import os
os.environ["OPENAI_API_KEY"] = "XXXXX"
os.environ["GOOGLE_API_KEY"] = "YYYYY"
os.environ["GOOGLE_CSE_ID"] = "ZZZZ"

In [16]:
from logging import getLogger
logger = getLogger(__name__)

In [17]:
def search_llm(query: str, model_name: str):
  llm = ChatOpenAI(temperature=0, model=model_name)
  google_search = GoogleSearchAPIWrapper()
  search_tools = [Tool(
      name = "Google Search",
      func=google_search.run,
      description="最新の話題について答える場合に利用することができます。また、今日の日付や今日の気温、天気、為替レートなど現在の状況についても確認することができます。入力は検索内容です。"
      ),]
  search_agent = initialize_agent(search_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
  search_result = search_agent.run(f"{query}")
  return search_result

# デモ化

In [18]:
def app(query: str, model_name: str):
  search_result = search_llm(query, model_name)
  logger.info(search_result)
  return search_result

In [19]:
title : str = "デモアプリ"
description : str = "このデモンストレーションは、Gradio（ユーザーインターフェースのライブラリ）を使用しています"
article : str='''このデモンストレーションは、ユーザーインターフェースを作成するためのPythonのライブラリであるGradioを使用しています。
'''

iface1 = gr.Interface(
    fn=app,
    inputs=[
        gr.Textbox(lines=1, placeholder="query Here"),
        gr.Dropdown(["gpt-3.5-turbo-16k", "gpt-4", "gpt-4-0613"]),
        ],
    outputs=["text"],
    cache_examples=True,
    examples=[
        ["ChatGPTでできること", "gpt-3.5-turbo-16k"],
        ["ChatGPTでできること", "gpt-4-0613"],
        ["日本で一番高い山", "gpt-4-0613"],
    ],
    title=title,
    description=description,
    article=article
    )
iface1.queue()
iface1.launch(share=True, auth=("user", "password"), debug=True)

Caching examples at: '/content/gradio_cached_examples/73'


> Entering new  chain...
I should search for information about the capabilities of ChatGPT.
Action: Google Search
Action Input: "ChatGPT capabilities"
Observation: Apr 11, 2023 ... It is capable of generating human-like text and can be used for a variety of applications. ... Here are some of ChatGPT's capabilities: Natural ... ChatGPT is a large language model developed by OpenAI. It is trained on a massive dataset of human-generated text and is able to generate human-like text in ... May 22, 2023 ... ChatGPT can be used for content creation, as it can easily write content based on a prompt. For instance, the AI tool can write a song based on ... You can ask a question, and ChatGPT answers it. · You can ask to generate a piece of text, for example an outline or a reflection on a particular topic. · You can ... Dec 10, 2022 ... ChatGPT can chat, answer questions, create content, write code and debug, take tests, manipulate data

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2ce40d803f8e55436b.gradio.live
